# 4. Assignment

## Portfolio Management
### MLP Training

#### January 2025

Questions?
* Contact: Mark Hendricks
* hendricks@uchicago.edu

***

# 1. Carry in Equities

### Data
You will need the files in the github repo, 
* `data/spx_carry_weekly.xlsx`
* `data/spx_returns_weekly.xlsx`

### 1. Data Processing
* Take the intersection of dates of the two data sets, so you have one unified set of dates.
* Report the maximum and minimum dividend yields from the entire panel of data. To which stocks did they belong?

In [120]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

SCALE = 52
dvd_yld_df = pd.read_excel('../data/spx_carry_weekly.xlsx', 's&p500 dvd yield').set_index('date')
ret_df = pd.read_excel('../data/spx_returns_weekly.xlsx', 's&p500 rets').set_index('date')

In [107]:
display(dvd_yld_df.head())
display(ret_df.head())

,A,AAPL,ABBV,ABT,ACN,ADI,ADM,ADP,AEE,AEP,AES,AFL,AIG,AIZ,AJG,ALB,ALL,ALLE,AMAT,AMCR,AME,AMGN,AMP,AMT,AON,AOS,APA,APD,APH,APO,APTV,ARE,ATO,AVB,AVGO,AVY,AWK,AXP,BA,BAC,...,TT,TXN,TXT,UDR,UHS,UNH,UNP,UPS,URI,USB,V,VICI,VLO,VLTO,VMC,VRSK,VST,VTR,VTRS,VZ,WAB,WBA,WDC,WEC,WELL,WFC,WM,WMB,WMT,WRB,WST,WTW,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZTS
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2015-01-02,0.009862,0.017196,0.029747,0.021381,0.022963,0.026647,0.018454,0.023532,0.035660,0.034669,0.029197,0.025540,0.008911,0.015936,0.030651,0.018239,0.015973,0.005813,0.016026,NaN,0.006842,0.019764,0.017564,0.015250,0.010601,0.010712,0.015667,0.022908,0.009321,0.123886,0.016447,0.032966,0.027907,0.027984,0.013987,0.027032,0.022989,0.011180,0.028011,0.011173,...,0.020377,0.025430,0.001897,0.033153,0.003605,0.014884,0.016862,0.024280,NaN,0.021860,0.007245,NaN,0.021851,NaN,0.003583,NaN,NaN,0.049219,NaN,0.046848,0.002755,0.017763,0.014489,0.031773,0.042885,0.025594,0.028986,0.050420,0.022352,0.008687,0.008436,0.026936,0.032124,0.040847,0.033223,0.029732,0.013445,0.031524,0.008057,0.007666
2015-01-09,0.009855,0.016784,0.029796,0.021239,0.022722,0.027047,0.019664,0.023179,0.036444,0.034393,0.031008,0.026608,0.009377,0.016280,0.030489,0.018638,0.015905,0.005727,0.016502,NaN,0.007060,0.020292,0.018170,0.015290,0.010551,0.010809,0.016477,0.023239,0.009328,0.122329,0.017017,0.031705,0.028235,0.026777,0.013347,0.026405,0.022836,0.011502,0.027672,0.011779,...,0.020875,0.025373,0.001879,0.032490,0.003545,0.014462,0.017394,0.024453,NaN,0.022812,0.007370,NaN,0.022610,NaN,0.003514,NaN,NaN,0.012653,NaN,0.047049,0.002828,0.017841,0.014258,0.031518,0.041515,0.026576,0.029109,0.053672,0.021489,0.008670,0.008496,0.026816,0.031763,0.040255,0.033167,0.029967,0.014476,0.031048,0.007674,0.007503
2015-01-16,0.010458,0.017738,0.030369,0.021578,0.022950,0.027566,0.020134,0.023176,0.035239,0.033349,0.029918,0.026813,0.009783,0.016570,0.031182,0.018797,0.015810,0.005996,0.017043,NaN,0.007414,0.019946,0.018611,0.015603,0.010465,0.010977,0.016064,0.023846,0.009538,0.121921,0.017929,0.030610,0.026680,0.025910,0.013485,0.026555,0.022258,0.012087,0.027833,0.013004,...,0.020919,0.025523,0.001866,0.031401,0.003696,0.014182,0.017873,0.024297,NaN,0.023603,0.007531,NaN,0.024229,NaN,0.003588,NaN,NaN,0.012338,NaN,0.045843,0.002883,0.017756,0.015042,0.030254,0.039141,0.027111,0.028714,0.054286,0.022127,0.008918,0.008484,0.027322,0.031668,0.040819,0.032415,0.030290,0.014853,0.031489,0.007753,0.007508
2015-01-23,0.010307,0.016640,0.031395,0.021888,0.022865,0.027151,0.020054,0.022568,0.035398,0.032858,0.031546,0.026562,0.009707,0.016544,0.031641,0.019353,0.015680,0.005874,0.016591,NaN,0.007373,0.019773,0.017918,0.015261,0.010591,0.010801,0.015903,0.023223,0.009106,0.118603,0.017137,0.030081,0.026594,0.025936,0.013087,0.026626,0.021924,0.012362,0.027039,0.012715,...,0.019936,0.024700,0.001888,0.030897,0.003756,0.013393,0.016654,0.026037,NaN,0.022622,0.007434,NaN,0.031898,NaN,0.003441,NaN,NaN,0.011965,NaN,0.046660,0.002824,0.017860,0.015080,0.029675,0.039649,0.026286,0.028302,0.052595,0.021692,0.008818,0.008679,0.027759,0.032186,0.041093,0.031780,0.030366,0.014558,0.030776,0.007754,0.007525
2015-01-30,0.010590,0.016046,0.032477,0.021448,0.024277,0.028404,0.020588,0.023749,0.036219,0.033753,0.032733,0.027330,0.010231,0.017005,0.033311,0.022793,0.016048,0.005925,0.017513,NaN,0.007516,0.020754,0.018569,0.015678,0.011105,0.012792,0.015982,0.022880,0.009309,0.117411,0.017371,0.030353,0.027412,0.028903,0.013608,0.026784,0.022088,0.012889,0.025040,0.013201,...,0.019413,0.025444,0.001880,0.031269,0.003901,0.014118,0.017063,0.027115,NaN,0.023383,0.007532,NaN,0.030257,NaN,0.003404,NaN,NaN,0.012058,NaN,0.048130,0.002876,0.018305,0.016456,0.030303,0.040268,0.026965,0.029166,0.051984,0.022594,0.008981,0.008923,0.027714,0.032357,0.040554,0.031974,0.031572,0.015015,0.031555,0.008092,0.007770


,A,AAPL,ABBV,ABT,ACGL,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIZ,AJG,AKAM,ALB,ALGN,ALL,ALLE,AMAT,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,AON,AOS,APA,APD,APH,APO,APTV,ARE,...,UAL,UDR,UHS,ULTA,UNH,UNP,UPS,URI,USB,V,VLO,VMC,VRSK,VRSN,VTR,VTRS,VZ,WAB,WAT,WBA,WDAY,WDC,WEC,WELL,WFC,WM,WMB,WMT,WRB,WST,WTW,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2016-01-15,-0.038352,0.001776,0.041190,0.003094,-0.028354,0.010896,0.015026,-0.003202,-0.083744,-0.002688,-0.102797,0.000000,0.007380,-0.064629,-0.001763,-0.023673,0.028920,-0.018018,-0.082121,-0.022444,-0.072369,-0.001717,-0.018238,-0.042290,-0.051402,-0.024046,0.003953,-0.022222,-0.061974,-0.060752,-0.058382,-0.005035,-0.010687,-0.055191,-0.036016,-0.010082,-0.024452,-0.044285,-0.083038,-0.063223,...,-0.119869,-0.005836,-0.028565,-0.064336,-0.008080,0.003385,-0.014774,-0.111793,-0.016374,-0.014407,-0.027102,-0.027942,-0.017855,-0.049904,-0.027905,0.005868,-0.008923,0.006809,-0.003788,-0.013817,-0.045026,-0.147137,0.024870,-0.025994,-0.014931,-0.015853,-0.207286,-0.025339,-0.053255,0.016397,-0.035071,-0.074553,0.010911,0.008844,0.038694,-0.040441,-0.008856,-0.004628,-0.045570,-0.054490
2016-01-22,0.023443,0.044184,0.025986,-0.012578,-0.017597,0.029919,0.005159,0.031313,0.054585,0.040940,-0.007490,-0.012871,-0.002726,0.073776,0.011656,0.004637,0.015332,0.001064,0.062514,0.070549,0.018209,0.006882,0.021814,0.060322,-0.004926,-0.020465,0.031589,0.004524,0.054980,0.045950,-0.008019,0.021537,0.003716,0.032906,0.114633,0.009760,0.070562,0.031389,-0.036017,-0.023204,...,0.022991,-0.001954,-0.015114,0.054444,0.046307,-0.055593,-0.001558,0.051729,0.008197,0.012112,0.021834,0.018243,-0.012766,0.012929,0.014811,0.068397,0.058744,0.009435,0.007684,0.011384,0.023574,-0.052710,0.016492,0.021562,0.004094,0.028338,0.226085,0.012274,-0.001015,0.030412,-0.005154,-0.015493,0.013707,0.003561,-0.013019,0.031835,0.028171,-0.019094,-0.000884,0.002658
2016-01-29,-0.008688,-0.040268,-0.066803,-0.054461,0.034298,0.032277,-0.005578,0.048268,0.063800,0.024411,-0.045074,0.045867,0.041689,0.048582,0.012049,0.002306,0.023154,-0.000266,-0.027499,0.026320,0.085152,0.035366,0.010007,0.035194,0.089109,0.002985,-0.021459,-0.027888,0.011688,-0.015728,-0.066831,0.016014,0.019995,0.064786,0.080243,0.065059,0.017868,-0.015941,-0.001537,0.011238,...,0.033390,-0.003361,0.047619,0.008012,0.007264,0.028720,0.036716,-0.170246,0.017525,0.024623,0.000147,0.033159,0.026144,0.005185,-0.003244,-0.007907,0.062286,-0.003738,-0.047166,-0.013855,-0.092990,0.072179,0.030026,-0.081625,0.024686,-0.000567,-0.022292,0.058542,0.019511,-0.012427,0.022510,0.007475,0.138075,0.043407,0.016717,0.056423,0.049144,0.001109,0.069027,-0.008065
2016-02-05,-0.042762,-0.028849,-0.032423,-0.011625,-0.023833,-0.062252,-0.111410,-0.075010,-0.036211,-0.021423,-0.013883,0.034951,0.030669,0.026312,0.004313,-0.055064,-0.034805,0.018068,-0.053047,-0.020896,-0.004082,0.045709,-0.070839,-0.047590,-0.100000,-0.036556,-0.050354,-0.086698,-0.058303,-0.144566,-0.013487,-0.039574,0.057384,-0.058101,-0.082041,0.070160,-0.000405,0.001472,-0.084848,-0.033214,...,-0.029412,-0.016018,-0.031960,-0.118452,-0.029873,0.042083,0.023273,0.006053,0.000749,-0.039604,-0.150139,0.061675,-0.069452,0.000926,0.018075,-0.024672,0.020011,-0.029399,-0.047356,-0.056697,-0.139184,-0.044186,0.032954,0.028352,-0.039811,-0.007178,-0.112957,0.009644,0.019146,-0.044740,-0.003931,-0.063255,-0.080634,0.034274,0.028646,0.019748,-0.036065,-0.042816,-0.050000,-0.049711
2016-02-12,0.004993,-0.000327,-0.010165,-0.007484,0.037305,-0.043144,-0.031944,-0.003612,-0.038858,0.005288,-0.027074,-0.026673,-0.026974,-0.059488,-0.008389,-0.006933,-0.145387,0.004435,0.175000,-0.003103,-0.107636,0.008522,0.021860,-0.044621,-0.075758,0.006616,0.004737,-0.023625,-0.035681,0.009838,-0.050334,-0.007556,0.010013,0.003812,-0.064533,-0.038717,-0.000202,-0.014706,0.035579,-0.061399,...,0.031797,-0.046541,-0.043837,-0.007639,0.000892,0.028922,0.01971

In [108]:
unified_dvd_yld_df = dvd_yld_df.loc[ret_df.index]
unified_dvd_yld_df

,A,AAPL,ABBV,ABT,ACN,ADI,ADM,ADP,AEE,AEP,AES,AFL,AIG,AIZ,AJG,ALB,ALL,ALLE,AMAT,AMCR,AME,AMGN,AMP,AMT,AON,AOS,APA,APD,APH,APO,APTV,ARE,ATO,AVB,AVGO,AVY,AWK,AXP,BA,BAC,...,TT,TXN,TXT,UDR,UHS,UNH,UNP,UPS,URI,USB,V,VICI,VLO,VLTO,VMC,VRSK,VST,VTR,VTRS,VZ,WAB,WBA,WDC,WEC,WELL,WFC,WM,WMB,WMT,WRB,WST,WTW,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZTS
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2016-01-15,0.012396,0.021415,0.039763,0.025654,0.022162,0.032116,0.035544,0.027207,0.039071,0.038167,0.051522,0.028970,0.019968,0.025553,0.039351,0.024212,0.020643,0.006817,0.024876,NaN,0.007517,0.026438,0.028870,0.022174,0.013939,0.011967,0.028305,0.029745,0.012310,0.104634,0.020527,0.038423,0.026679,0.028458,0.014670,0.025110,0.022487,0.018439,0.034705,0.013831,...,0.028900,0.031647,0.002142,0.031023,0.003664,0.018303,0.029686,0.032430,NaN,0.026120,0.007796,NaN,0.030116,NaN,0.004771,NaN,NaN,0.053392,NaN,0.050867,0.005032,0.018016,0.042337,0.037536,0.049759,0.030725,0.029891,0.159006,0.031649,0.009748,0.008536,0.029190,0.048025,0.034264,0.035068,0.037639,0.017077,0.038142,0.008974,0.008760
2016-01-22,0.012112,0.020509,0.038756,0.025981,0.021518,0.031141,0.033704,0.026137,0.039581,0.038271,0.047983,0.028636,0.019876,0.025167,0.039309,0.022616,0.020502,0.006671,0.023460,NaN,0.007674,0.025628,0.028740,0.021019,0.013887,0.011585,0.025394,0.029458,0.011499,0.101449,0.021294,0.039336,0.026242,0.028681,0.014135,0.024366,0.021781,0.021068,0.034989,0.014749,...,0.029633,0.030219,0.002136,0.031084,0.003720,0.017493,0.031433,0.032481,NaN,0.025908,0.007703,NaN,0.035367,NaN,0.004685,NaN,NaN,0.052613,NaN,0.048044,0.004985,0.017813,0.044693,0.036927,0.048708,0.030600,0.029068,0.129686,0.031265,0.009758,0.008284,0.029341,0.048780,0.033801,0.034944,0.038135,0.016550,0.037097,0.009149,0.008756
2016-01-29,0.012218,0.021368,0.041530,0.027477,0.020845,0.029707,0.031683,0.025515,0.037845,0.036739,0.046316,0.028295,0.019830,0.024597,0.040383,0.022036,0.019802,0.006605,0.022663,NaN,0.007651,0.026190,0.029564,0.020776,0.013663,0.013744,0.023507,0.027658,0.011297,0.103093,0.021326,0.038899,0.024270,0.029156,0.013163,0.024306,0.020952,0.021682,0.036294,0.014144,...,0.029051,0.028717,0.002338,0.031189,0.003551,0.017367,0.030556,0.031330,NaN,0.025462,0.007518,NaN,0.035362,NaN,0.004535,NaN,NaN,0.052784,NaN,0.045227,0.005004,0.018063,0.041684,0.035850,0.055288,0.029863,0.029084,0.132642,0.029536,0.009571,0.008389,0.028695,0.048419,0.029700,0.033490,0.037508,0.015666,0.035359,0.009139,0.008827
2016-02-05,0.012764,0.022123,0.042922,0.027800,0.022229,0.032116,0.035222,0.026073,0.036567,0.035646,0.045128,0.028174,0.020986,0.025484,0.039666,0.022507,0.018936,0.008530,0.023795,NaN,0.007942,0.027579,0.032371,0.022062,0.012921,0.014643,0.025608,0.025845,0.011302,0.082353,0.023304,0.040235,0.024162,0.032477,0.013711,0.023418,0.021343,0.021489,0.035574,0.015444,...,0.032138,0.030473,0.002477,0.031696,0.003668,0.017902,0.029322,0.030618,NaN,0.025443,0.007828,NaN,0.042017,NaN,0.004272,NaN,NaN,0.051847,NaN,0.044340,0.005155,0.019149,0.043611,0.034706,0.054499,0.031341,0.029294,0.149533,0.029254,0.009392,0.008782,0.016839,0.051688,0.032305,0.032380,0.036464,0.016901,0.036682,0.009547,0.009289
2016-02-12,0.012700,0.022130,0.043362,0.028010,0.023231,0.032232,0.036980,0.025936,0.037569,0.036964,0.047983,0.028616,0.024151,0.029820,0.039491,0.022577,0.020654,0.008348,0.024907,NaN,0.007890,0.027640,0.033437,0.022878,0.012793,0.014587,0.027375,0.026886,0.011304,0.083582,0.022615,0.042867,0.024058,0.032843,0.014467,0.023132,0.020897,0.022028,0.040136,0.016736,...,0.032150,0.029722,0.002494,0.033243,0.003837,0.017886,0.028497,0.032082,NaN,0.026087,0.007952,NaN,0.041558,NaN,0.008694,NaN,NaN,0.060293,NaN,0.045101,0.004897,0.018682,0.048685,0.035567,0.063235,0.031706,0.028529,0.190052,0.029616,0.009340,0.008699,0.017804,0.054867,0.028927,0.032973,0.036036,0.016728,0.038000,0.009883,0.009399
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

### 2. Static Carry
* Starting at the beginning of the sample, build a portfolio where you go (equally) long the top 100 stocks as ranked by dividend yield, and you short (equally) the smallest 100 stocks as ranked by dividend yield.
* Using those weights, calculate the return of your portfolio over time. 

Report the following annualized stats
* mean
* volatility
* Sharpe

In [109]:
# get the first row
first_row =  dvd_yld_df.iloc[0].sort_values()

# pick losers and winners
losers = first_row[0:100].index
winners = first_row[-100:].index
remaining = first_row.loc[~first_row.index.isin(losers) & ~first_row.index.isin(winners)].index

# replicate positions
positions = first_row.copy()
positions[losers] = 0.01
positions[winners] = -0.01
positions[remaining] = 0.0

positions.name = "Static Positions"
positions.head()


COO     0.01
CI      0.01
C       0.01
GPN     0.01
FICO    0.01
Name: Static Positions, dtype: float64

In [110]:
positions.sort_index()

A       0.01
AAPL    0.00
ABBV    0.00
ABT     0.00
ACN     0.00
        ... 
XOM     0.00
XYL     0.01
YUM     0.00
ZBH     0.01
ZTS     0.01
Name: Static Positions, Length: 416, dtype: float64

In [111]:
ret_df.columns

Index(['A', 'AAPL', 'ABBV', 'ABT', 'ACGL', 'ACN', 'ADBE', 'ADI', 'ADM', 'ADP',
       ...
       'WTW', 'WY', 'WYNN', 'XEL', 'XOM', 'XYL', 'YUM', 'ZBH', 'ZBRA', 'ZTS'],
      dtype='object', length=465)

In [112]:
positions.index

Index(['COO', 'CI', 'C', 'GPN', 'FICO', 'TXT', 'TPL', 'CTRA', 'WAB', 'EQT',
       ...
       'STZ', 'SW', 'SYF', 'TMUS', 'URI', 'VICI', 'VLTO', 'VRSK', 'VST',
       'VTRS'],
      dtype='object', length=416)

In [116]:
common_cols = list(set(ret_df.columns) & set(positions.index))
filtered_ret_df = ret_df[common_cols]
filtered_ret_df = filtered_ret_df.reindex(sorted(filtered_ret_df.columns), axis=1)
filtered_positions = positions[common_cols]

filtered_ret_df.shape, filtered_positions.shape

((468, 392), (392,))

In [119]:
port_ret = filtered_ret_df.dot(filtered_positions)
port_ret.name = 'Return'

In [130]:
port_mean_ret = port_ret.mean() * SCALE
port_mean_ret

np.float64(0.05053491764443579)

In [131]:
port_vol = port_ret.std() * np.sqrt(SCALE)
port_vol

np.float64(0.06896267209389502)

In [132]:
port_sharpe_ratio = port_mean_ret / port_vol
port_sharpe_ratio = "Sharpe Ratio"
port_sharpe_ratio

'Sharpe Ratio'

### 3. Challenges

Mention two challenges of this strategy.

### 4. Dynamic Carry

Make the carry strategy of `2` dynamic by recalculating the weights every period.

Report the performance statistics.

### 5. Market Exposure

Calculate a linear factor decomposition of your carry strategy (from part `2` or part `4`) against `SPY`.

Report the...
* alpha (annualized)
* beta
* r-squared

***